# Keyphrase similality

Date: 2024/06/15

The ChromaDB's default embedder "sentence-transformers/all-MiniLM-L6-v2"

Choose a multi-lingual one for Japanese:

https://docs.trychroma.com/guides/embeddings

In [1]:
import spacy
import textacy
from textacy import extract

In [2]:
text = open("beatles.txt").read()

In [3]:
nlp = spacy.load("ja_core_news_lg")
doc = nlp(text)

/Users/shiny/Documents/GitHub/learning-langchain/venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
sents = [sent.text for sent in doc.sents]

In [5]:
en = textacy.load_spacy_lang("ja_core_news_lg")

In [17]:
kps = []
for sent in sents:
    doc = textacy.make_spacy_doc(sent, lang=en)
    kps_textrank = [kps.replace(' ', '') for kps, _ in extract.keyterms.textrank(doc, normalize="lemma", topn=10)]
    #print(kps_textrank)
    kps.extend(kps_textrank)

In [18]:
kps

['ロックバンド',
 '年代',
 'イギリス',
 '人気',
 '世界',
 'ビートルズ',
 'ringostarr',
 'PaulMcCartney',
 'JOHNlennon',
 'GeorgeHarrison',
 '人',
 'ジョン',
 'リンゴ',
 'ポール',
 'ジョージ',
 'レノン',
 '音楽史',
 '多様性',
 '革新性',
 'ハードロック',
 'サイケデリックロック',
 '大きな影響',
 '多岐',
 'ポップ',
 '結成',
 '活動',
 'ビートルズ',
 '初期',
 'リヴァプール',
 'ベスト',
 'レノン',
 'ピート',
 'メンバー',
 'ドラマー',
 'マッカートニー',
 'ハリスン',
 '当初',
 '演奏活動',
 'ハンブルク',
 '過酷',
 'スキル',
 'クラブ',
 '結束力',
 '期間',
 '音楽',
 'バンド',
 '成長',
 '年',
 'バンド',
 'ベスト',
 '代わり',
 'スター',
 '勧め',
 'プロデューサー',
 'マーティン',
 'ジョージ',
 'LOVEMEdo',
 'デビューシングル',
 'ブレイクスルー',
 'ドゥ',
 '始まり',
 'ラヴ',
 '年',
 'ビートルズ',
 '月',
 'ミー',
 'FromMEtoYOU',
 'PleasePleaseME',
 'ShelovesYOU',
 'イギリス国内',
 'ヒット曲',
 '年',
 '爆発',
 '人気',
 'ユー',
 'ミー',
 'ビートルマニア',
 '進出',
 'アメリカ',
 '出演',
 '年',
 'エド',
 'ショー',
 'ビートルズ',
 'サリヴァン',
 'アメリカ国内',
 '一大センセーション',
 'ビートルマニア',
 'パフォーマンス',
 '現象',
 'IwanttoHoldYourHand',
 'ビルボードチャート',
 'トップスター',
 '抱く',
 'シングル',
 'アメリカ',
 '位',
 '座',
 '大きな進化',
 '年',
 '音楽',
 'ビートルズ',
 '高い評価',
 '音楽要素',
 'フォークロック',
 'rubberSoul',
 '作

In [19]:
import chromadb
from chromadb.utils import embedding_functions

multi = embedding_functions.SentenceTransformerEmbeddingFunction(model_name="distiluse-base-multilingual-cased-v2")

chroma_client = chromadb.Client()

In [20]:
collection = chroma_client.create_collection(name="collection-beatles", embedding_function=multi)

In [21]:
ids = [str(i) for i in range(len(kps))]

In [22]:
collection.add(
    documents=kps,
    ids=ids
)

In [29]:
results = collection.query(
    query_texts=["若者", "インド", "移住", "成功"],
    n_results=5 # how many results to retur
)
print(results)

{'ids': [['10', '253', '169', '314', '185'], ['151', '2', '95', '78', '87'], ['77', '253', '169', '107', '42'], ['221', '135', '42', '50', '36']], 'distances': [[0.22731176018714905, 0.27179038524627686, 0.27179038524627686, 0.2774984836578369, 0.2774984836578369], [0.0, 0.4416681230068207, 0.4966648817062378, 0.4966648817062378, 0.5327897071838379], [0.2359115481376648, 0.28412696719169617, 0.28412696719169617, 0.31656327843666077, 0.31763333082199097], [0.0, 0.24527516961097717, 0.29833003878593445, 0.3079608678817749, 0.30821317434310913]], 'metadatas': [[None, None, None, None, None], [None, None, None, None, None], [None, None, None, None, None], [None, None, None, None, None]], 'embeddings': None, 'documents': [['人', '様々', '様々', 'ザ', 'ザ'], ['インド', 'イギリス', 'アメリカ', 'アメリカ', 'ビートルマニア'], ['進出', '様々', '様々', 'ラバー', '結束力'], ['成功', '完成度', '結束力', '代わり', '当初']], 'uris': None, 'data': None}
